In [70]:
import os

import numpy as np

from photutils.utils import ImageDepth
from photutils.segmentation import SourceFinder

from astropy.io import fits

import matplotlib.pyplot as plt

In [71]:
def measure():

    '''
    This function calculates the depth of the F275W observations of the Sunburst Arc
    '''

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data/'
    
    # Set the file path to the F275W observation
    f275w_file = f'{data}/hst/V5.0_PSZ1G311.65-18.48_F275W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'

    # Get the inverse sensitivity of the image from the FITS header of the file
    inverse_sensitivity = fits.getheader(f275w_file)['PHOTFLAM']

    # Get the data array of the F275W observation for a core area of the drizzled image, where all the
    # frames commonly overlap. We want to calculate the depth of the image from this portion of the image
    # because this is where the Sunburst Arc also lies, so it is the best representation of the depth
    # achieved on the Sunburst Arc
    f275w = fits.getdata(f275w_file)[3500:5000, 3500:5500]

    # Convert the data from counts/s to erg/s/cm^2/Å
    f275w = f275w * inverse_sensitivity

    # Convert the data to a frequency-space flux density (erg/s/cm^2/Hz) based on the pivot wavelength of
    # the WFC3/UVIS F275W filter
    f275w = f275w * 2709.7**2 / (3e8 * 1e10)

    # Create a mask representing the location of any NaNs in the image
    nan_mask = np.isnan(f275w)

    # Create a mask of the sources in the image
    finder = SourceFinder(npixels=9, progress_bar=False)
    mask = finder(np.where(nan_mask, 0, f275w), abs(np.nanmedian(f275w))).make_source_mask()

    # Determine the depth of the image at a significance of 5σ, using AB magnitudes
    depth = ImageDepth(4., nsigma=5., mask_pad=4., napers=100, niters=1000, seed=123, zeropoint=-48.6, progress_bar=False)
    limits = depth(f275w, (nan_mask | mask))

    print(limits)

In [72]:
measure()

(2.042339460023522e-31, 28.12468017893024)
